# SQL Frequently Asked Interview Queries 
Most are from leetcode. 


### Write an SQL query that makes recommendations using the pages that your friends liked. Assume you have two tables: a two-column table of users and their friends, and a two-column table of users and the pages they liked. It should not recommend pages you already like.

```SQL
SELECT DISTINCT page_id AS recommended_page 
FROM Likes 
WHERE user_id IN (
    SELECT user2_id AS friend_id FROM Friendship WHERE user1_id = 1
    UNION
    SELECT user1_id AS friend_id FROM Friendship WHERE user2_id = 1
) AND page_id NOT IN (
    SELECT page_id FROM Likes WHERE user_id = 1
); 
```


### Given two tables Friend_request (requester_id, sent_to_id, time) and Request_accepted (accepter_id, requester_id, time), write a query to find the overall acceptance rate of requests rounded to 2 decimals, which is the number of acceptance divide the number of requests.

```SQL
SELECT IFNULL(round((SELECT count(distinct concat(requester_id, accepter_id)) 
                     FROM request_accepted)/(SELECT count(distinct concat(sender_id, send_to_id)) 
                     FROM friend_request), 2), 0) AS accept_rate; 
```


### Given two tables Friend_request (requester_id, sent_to_id, time) and Request_accepted (accepter_id, requester_id, time), Write a query to find the the people who has most friends and the most friends. 

```SQL
SELECT id2 AS id, count(id1) AS num 
FROM (
    SELECT requester_id id1, accepter_id id2 FROM request_accepted 
    UNION 
    SELECT accepter_id id1, requester_id id2 FROM request_accepted 
) t1
GROUP BY id2
ORDER BY num DESC
LIMIT 1; 
```


### Write an SQL query that reports for every date within at most 90 days from today, the number of users that logged in for the first time on that date. Assume today is 2019-06-30.

```SQL
SELECT activity_date AS login_date, COUNT(user_id) AS user_count 
FROM (
    SELECT user_id, MIN(activity_date) activity_date 
    FROM Traffic 
    WHERE activity = 'login'
    GROUP BY 1
) t1 
WHERE DATEDIFF('2019-06-30', activity_date) <= 90 
GROUP BY 1; 
```


### Write an SQL query to find the click-through-rate of each Ad. 

```SQL
SELECT ad_id, ROUND(IFNULL(
                AVG(CASE 
                    WHEN action = 'clicked' THEN 1 
                    WHEN action = 'viewed' THEN 0 
                    ELSE NULL END)*100,0),2) AS ctr 
FROM ads 
GROUP BY ad_id 
ORDER BY ctr DESC, ad_id ASC; 
```


### Write an SQL query to find number of comments per each post. 

```SQL
SELECT s1.sub_id AS post_id, COUNT(DISTINCT s2.sub_id) AS number_of_comments 
FROM submissions s1 
LEFT JOIN submissions s2 
ON s1.sub_id = s2.parent_id
WHERE s1.parent_id IS NULL
GROUP BY post_id; 

```


### Write an SQL query to get the amount of each follower’s follower if he/she has one.

```SQL
SELECT f2.followee AS follower, COUNT(DISTINCT f2.follower) AS num 
FROM follow f1
JOIN follow f2 
ON f1.follower = f2.followee
GROUP BY 1;
```


### Write an SQL query to get the second and a function to get the N-th highest salary. 

```SQL
SELECT MAX(salary) AS SecondHighestSalary 
FROM salary 
WHERE salary < (SELECT MAX(salary) FROM salary);
```

---
```SQL
CREATE FUNCTION getNthHighestSalary(N INT) RETURNS INT
BEGIN
    DECLARE M INT; 
    SET M=N-1;
    RETURN (
          SELECT distinct salary
          FROM employee
          ORDER BY salary DESC 
          LIMIT M,1
      );
END
```


### Write a SQL query to find the cancellation rate of requests made by unbanned users (both client and driver must be unbanned) between Oct 1, 2013 and Oct 3, 2013. The cancellation rate is computed by dividing the number of canceled (by client or driver) requests made by unbanned users by the total number of requests made by unbanned users. 

```SQL
SELECT t1.Request_at Day,
    ROUND(SUM(CASE WHEN t1.status = 'completed' THEN 0 ELSE 1 END) / COUNT(*),2) AS 'Cancellation Rate'
FROM (
    SELECT * 
    FROM Users u 
    JOIN Trips t
    ON u.Users_Id = t.Client_Id
    WHERE u.Banned = 'No'
)t1
WHERE t1.Request_at BETWEEN '2013-10-01' AND '2013-10-03'
GROUP BY 1;
```

--- 
```SQL
SELECT t1.request_at as Day, 
    ROUND(AVG(CASE WHEN t1.status LIKE 'cancelled_by%' THEN 1 ELSE 0 END),2) AS 'Cancellation Rate'
FROM (
    SELECT * 
    FROM trips t 
    JOIN users u 
    ON t.client_id = u.users_id 
    WHERE u.banned = 'No' 
) t1
WHERE request_at BETWEEN '2013-10-01' AND '2013-10-03' 
GROUP BY 1; 
```


### Write a SQL query to rank scores. If there is a tie between two scores, both should have the same ranking. Note that after a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no "holes" between ranks.

```SQL

```


### Pivot the Occupation column in OCCUPATIONS so that each Name is sorted alphabetically and displayed underneath its corresponding Occupation. The output column headers should be Doctor, Professor, Singer, and Actor, respectively. 
https://www.hackerrank.com/challenges/occupations/problem

```SQL
SET @r1=0, @r2=0, @r3=0, @r4=0;
SELECT min(Doctor), min(Professor), min(Singer), min(Actor)
FROM (
    SELECT case when Occupation='Doctor' then (@r1:=@r1+1)
                when Occupation='Professor' then (@r2:=@r2+1)
                when Occupation='Singer' then (@r3:=@r3+1)
                when Occupation='Actor' then (@r4:=@r4+1) end as RowNumber,
        case when Occupation='Doctor' then Name end as Doctor,
        case when Occupation='Professor' then Name end as Professor,
        case when Occupation='Singer' then Name end as Singer,
        case when Occupation='Actor' then Name end as Actor
    FROM OCCUPATIONS
    ORDER BY Name
) Temp
GROUP BY RowNumber;

```


### Query the median of the Northern Latitudes (LAT_N) from STATION and round your answer to  decimal places. 
https://www.hackerrank.com/challenges/weather-observation-station-20/problem

```SQL
SELECT round(S.LAT_N,4) mediam 
FROM station S 
WHERE (SELECT count(Lat_N) FROM station WHERE Lat_N < S.LAT_N) = 
        (SELECT count(Lat_N) FROM station WHERE Lat_N > S.LAT_N); 

```


10. The success ratio of sending messages given sent and receive table. 


11. How to keep a nightly count of songs listened by users based on a real-time transaction table. 
	
